## Exercise:
- Upload sales.csv and products.csv to **bronze**
- Create **silver** database and tables for both sales and products (take bellow transformations into consideration)
- Load product **silver** table
- Load sales **silver** table, taking this into consideration: 
    1. Remove duplicates
    2. Filter total_price < 1000
    3. Create the columns total_cost and revenue


- Create **gold** database and tables:
    1. Filter sales where payment_type != "voucher"
    2. Create and load gold flat table aggregated by country, year, product_name, product_color (no need for dimensions)
    3. (Similar to 2.) Create and load a gold conformed dimension product and create and load a gold flat table aggregated by country, year, product

In [ ]:
# Install dependencies

import sys
!{sys.executable} -m pip install hdfs

In [12]:
pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

## Bronze

In [88]:
# upload sales and products to bronze

from os import PathLike
from hdfs import InsecureClient
client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

sales_from_path = "./sales.csv"
sales_to_path = "/demo/bronze/sales.csv"
products_from_path = "./products.csv"
products_to_path = "/demo/bronze/products.csv"

client.delete(products_to_path)
client.delete(sales_to_path)

client.upload(sales_to_path, sales_from_path)
client.upload(products_to_path, products_from_path)

# More HDFS functions here if needed (e.g., read, write, list files, delete files, rename, delete, etc...)
# https://hdfscli.readthedocs.io/en/latest/quickstart.html

'/demo/bronze/products.csv'

## Silver

In [60]:
# create silver database
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS silver_sales_demo LOCATION 'hdfs://hdfs-nn:9000/silver/silver_sales_demo.db/'
    """
)

# create products table in silver
spark.sql(
    """
    DROP TABLE IF EXISTS silver_sales_demo.products
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE silver_sales_demo.products (
        product_id INT,
        name STRING,
        color STRING,
        family STRING,
        desc STRING, 
        type STRING
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/silver/silver_sales_demo.db/products/'
    """
)

# create sales table in silver
spark.sql(
    """
    DROP TABLE IF EXISTS silver_sales_demo.sales
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE silver_sales_demo.sales (
        customer STRING,
        order_date INT,
        country STRING,
        payment_type STRING,
        product INT,
        quantity INT, 
        total_price INT,
        unit_price INT,
        unit_cost INT,
        total_cost INT,
        revenue INT
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/silver/silver_sales_demo.db/sales/'
    """
)



21/11/28 20:33:44 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `silver_sales_demo`.`products` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
21/11/28 20:33:48 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `silver_sales_demo`.`sales` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [61]:
spark.sql("USE silver_sales_demo")
spark.sql("SHOW tables").show()

+-----------------+---------+-----------+
|         database|tableName|isTemporary|
+-----------------+---------+-----------+
|silver_sales_demo| products|      false|
|silver_sales_demo|    sales|      false|
+-----------------+---------+-----------+



In [89]:
from pyspark.sql.functions import col

# read sales 

sales_df = spark \
    .read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv("hdfs://hdfs-nn:9000/demo/bronze/sales.csv")

# apply filters and enhancements
sales_df = sales_df \
    .filter("total_price < 1000") \
    .withColumn("total_cost", col("unit_cost") * col("quantity")) \
    .withColumn("revenue", col("total_price") - col("total_cost")) \
    .drop_duplicates()

# write to delta table
sales_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/silver/silver_sales_demo.db/sales/")

# check the results in the table
spark.table("silver_sales_demo.sales").show()
spark.table("silver_sales_demo.sales").count()

21/11/28 21:07:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
21/11/28 21:07:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
21/11/28 21:07:48 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


+--------+----------+--------+------------+-------+--------+-----------+----------+---------+----------+-------+
|customer|order_date| country|payment_type|product|quantity|total_price|unit_price|unit_cost|total_cost|revenue|
+--------+----------+--------+------------+-------+--------+-----------+----------+---------+----------+-------+
|  cust11|  20211009|Portugal|        cash|      1|       3|         90|        30|       15|        45|     45|
|   cust7|  20211016| Belgium|        cash|      6|       4|        128|        32|       20|        80|     48|
|  cust99|  20211018| Belgium|        cash|      7|       1|         50|        50|       20|        20|     30|
|   cust2|  20211010|Portugal|     voucher|      4|       3|        105|        35|       12|        36|     69|
|  cust11|  20211009|Portugal|        card|      4|       1|         40|        40|       20|        20|     20|
|  cust11|  20211009|Portugal|        card|      3|       1|         40|        40|       20|   

20

In [90]:
from pyspark.sql.functions import col

# read products 

products_df = spark \
    .read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv("hdfs://hdfs-nn:9000/demo/bronze/products.csv")

# write to delta table
products_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/silver/silver_sales_demo.db/products/")

# check the results in the table
spark.table("silver_sales_demo.products").show()
spark.table("silver_sales_demo.products").count()

+----------+----------+---------------+------+-------------------+--------+
|product_id|      name|          color|family|               desc|    type|
+----------+----------+---------------+------+-------------------+--------+
|         1|      stan|          black|  fam1|           one desc|sneakers|
|         2|      stan|            red|  fam2|     different desc|sneakers|
|         3|   adizero|white and black|  fam3|           adi desc|sneakers|
|         4|  predator|          green|  fam4|          pred desc|sneakers|
|         5|    terrex|     blue ocean|  fam5|        terrex desc|sneakers|
|         6|    terrex|          brown|  fam6|terrex amazing desc|sneakers|
|         7| superstar|          black|  fam7|         super desc|sneakers|
|         8|    falcon|          brown|  fam8|        falcon desc|sneakers|
|         9|ultraboost|            red|  fam9|         ultra desc|sneakers|
|        10|     yeezy|         yellow| fam10|         yeezy desc|sneakers|
+----------+

10

## GOLD Flat Table

In [96]:
# create gold database
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS gold_sales_demo LOCATION 'hdfs://hdfs-nn:9000/gold/gold_sales_demo.db/'
    """
)

# create sales_per_country table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS gold_sales_demo.sales_per_country
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE gold_sales_demo.sales_per_country (
        year INT,
        country STRING,
        product_name STRING,
        product_color STRING,
        quantity INT, 
        total_price INT,
        unit_price_avg DOUBLE,
        unit_cost_avg DOUBLE,
        total_cost INT,
        revenue INT
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/gold/gold_sales_demo.db/sales_per_country/'
    """
)

21/11/28 21:11:35 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `gold_sales_demo`.`sales_per_country` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [103]:
from pyspark.sql.functions import substring, avg, sum

# read data from the silver tables
products_df = spark.table("silver_sales_demo.products")
sales_df = spark.table("silver_sales_demo.sales")

# join to make a flat table with product information
flat_sales_df = sales_df \
    .join(products_df, sales_df.product == products_df.product_id) \
    .withColumn("year", substring(col("order_date"), 1, 4).cast('INT')) \
    .withColumnRenamed("name", "product_name") \
    .withColumnRenamed("color", "product_color") \
    .select("year", "country", "product_name", "product_color", "quantity", 
            "total_price", "unit_price", "unit_cost", "total_cost", "revenue")

# aggregate data per country, year and product (name and color)
sales_per_country_df = flat_sales_df \
    .filter("payment_type != 'voucher'") \
    .groupBy("year", "country", "product_name", "product_color") \
    .agg(
        sum(flat_sales_df.quantity).cast('INT').alias("quantity"),
        sum(flat_sales_df.total_price).cast('INT').alias("total_price"),
        avg(flat_sales_df.unit_cost).alias("unit_cost_avg"),
        avg(flat_sales_df.unit_price).alias("unit_price_avg"),
        sum(flat_sales_df.total_cost).cast('INT').alias("total_cost"),
        sum(flat_sales_df.revenue).cast('INT').alias("revenue"),
    )

# write to delta table
sales_per_country_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/gold/gold_sales_demo.db/sales_per_country/")

# check the results in the table
spark.table("gold_sales_demo.sales_per_country").show()
spark.table("gold_sales_demo.sales_per_country").count()

21/11/28 21:18:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


+----+--------+------------+---------------+--------+-----------+------------------+-------------+----------+-------+
|year| country|product_name|  product_color|quantity|total_price|    unit_price_avg|unit_cost_avg|total_cost|revenue|
+----+--------+------------+---------------+--------+-----------+------------------+-------------+----------+-------+
|2021|Portugal|     adizero|white and black|       4|        302|63.666666666666664|         20.0|        80|    222|
|2021|Portugal|      terrex|     blue ocean|       5|        175|              35.0|         15.0|        75|    100|
|2021|Portugal|   superstar|          black|       2|         80|              40.0|         20.0|        40|     40|
|2021|Portugal|    predator|          green|       4|        175|              42.5|         20.0|        80|     95|
|2021| Belgium|   superstar|          black|      12|        500|              45.0|         20.0|       240|    260|
|2021| Belgium|  ultraboost|            red|       4|   

11

## Gold Flat Table with Product Conformed Dimension

In [112]:
# create gold database
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS gold_sales_demo LOCATION 'hdfs://hdfs-nn:9000/gold/gold_sales_demo.db/'
    """
)

# create dim_product conformed dimension table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS gold_sales_demo.dim_product
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE gold_sales_demo.dim_product (
        product_id INT,
        name STRING,
        color STRING,
        family STRING,
        desc STRING, 
        type STRING
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/gold/gold_sales_demo.db/dim_product/'
    """
)

# create sales_per_country_conformed table in gold
spark.sql(
    """
    DROP TABLE IF EXISTS gold_sales_demo.sales_per_country_conformed
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE gold_sales_demo.sales_per_country_conformed (
        year INT,
        country STRING,
        product INT,
        quantity INT, 
        total_price INT,
        unit_price_avg DOUBLE,
        unit_cost_avg DOUBLE,
        total_cost INT,
        revenue INT
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/gold/gold_sales_demo.db/sales_per_country_conformed/'
    """
)

21/11/28 21:26:57 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `gold_sales_demo`.`dim_product` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
21/11/28 21:26:58 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `gold_sales_demo`.`sales_per_country_conformed` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


DataFrame[]

In [114]:
# create Product dimension

# read data from the silver tables
products_df = spark.table("silver_sales_demo.products")

# write to delta table
products_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/gold/gold_sales_demo.db/dim_product/")

# check the results in the table
spark.table("gold_sales_demo.dim_product").show()
spark.table("gold_sales_demo.dim_product").count()

+----------+----------+---------------+------+-------------------+--------+
|product_id|      name|          color|family|               desc|    type|
+----------+----------+---------------+------+-------------------+--------+
|         1|      stan|          black|  fam1|           one desc|sneakers|
|         2|      stan|            red|  fam2|     different desc|sneakers|
|         3|   adizero|white and black|  fam3|           adi desc|sneakers|
|         4|  predator|          green|  fam4|          pred desc|sneakers|
|         5|    terrex|     blue ocean|  fam5|        terrex desc|sneakers|
|         6|    terrex|          brown|  fam6|terrex amazing desc|sneakers|
|         7| superstar|          black|  fam7|         super desc|sneakers|
|         8|    falcon|          brown|  fam8|        falcon desc|sneakers|
|         9|ultraboost|            red|  fam9|         ultra desc|sneakers|
|        10|     yeezy|         yellow| fam10|         yeezy desc|sneakers|
+----------+

10

21/11/29 02:15:28 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 8587446 ms exceeds timeout 120000 ms
21/11/29 02:15:28 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:996)
	at org.apache.spark.e

In [111]:
from pyspark.sql.functions import substring, avg, sum

# read data from the silver tables
sales_df = spark.table("silver_sales_demo.sales")

# aggregate data per country, year and product
sales_per_country_df = sales_df \
    .withColumn("year", substring(col("order_date"), 1, 4).cast('INT')) \
    .filter("payment_type != 'voucher'") \
    .groupBy("year", "country", "product") \
    .agg(
        sum(sales_df.quantity).cast('INT').alias("quantity"),
        sum(sales_df.total_price).cast('INT').alias("total_price"),
        avg(sales_df.unit_cost).alias("unit_cost_avg"),
        avg(sales_df.unit_price).alias("unit_price_avg"),
        sum(sales_df.total_cost).cast('INT').alias("total_cost"),
        sum(sales_df.revenue).cast('INT').alias("revenue"),
    )

# write to delta table
sales_per_country_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/gold/gold_sales_demo.db/sales_per_country_conformed/")

# check the results in the table
spark.table("gold_sales_demo.sales_per_country_conformed").show()
spark.table("gold_sales_demo.sales_per_country_conformed").count()

21/11/28 21:25:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


+----+--------+-------+--------+-----------+------------------+-------------+----------+-------+
|year| country|product|quantity|total_price|    unit_price_avg|unit_cost_avg|total_cost|revenue|
+----+--------+-------+--------+-----------+------------------+-------------+----------+-------+
|2021|Portugal|      5|       5|        175|              35.0|         15.0|        75|    100|
|2021|Portugal|      7|       2|         80|              40.0|         20.0|        40|     40|
|2021| Belgium|      7|      12|        500|              45.0|         20.0|       240|    260|
|2021|Portugal|      4|       4|        175|              42.5|         20.0|        80|     95|
|2021|Portugal|      1|       6|        180|              30.0|         15.0|        90|     90|
|2021|Portugal|      3|       4|        302|63.666666666666664|         20.0|        80|    222|
|2021|Portugal|      2|       1|         41|              41.0|         20.0|        20|     21|
|2021| Belgium|     10|      1

11

#### Final Notes:
As you can see, the drawback of using a Conformed Dimension in this case, would be the fact that you would need to perform a join between your fact table and you dimension if you wanted to check further details of the Product. It wouldn't be immediate feedback. 
However, there are also some advantages as the fact that the same table can be shared across different fact tables, ensuring the company follows the same picture. 